In [ ]:
!pip install transformers[torch]

In [ ]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv('Mental-Health-Twitter.csv')

# Split into train and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Load the ALBERT tokenizer
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

# Tokenize the tweet text
train_encodings = tokenizer(train_data['post_text'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_data['post_text'].tolist(), truncation=True, padding=True)

# Create PyTorch datasets
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_data['label'].tolist())
val_dataset = TweetDataset(val_encodings, val_data['label'].tolist())

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

In [ ]:
# Load the ALBERT model
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=2)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=100
)

# Create the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

Step,Training Loss
500,0.507400
1000,0.376800
1500,0.304600
2000,0.265800
2500,0.203500
3000,0.177500


{'eval_loss': 0.33063414692878723,
 'eval_runtime': 19.361,
 'eval_samples_per_second': 206.6,
 'eval_steps_per_second': 3.254,
 'epoch': 3.0}